In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [85]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df.loc[:, 'term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df.loc[:, 'length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df.loc[:, 'rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df.loc[:, 'room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df.loc[:, 'n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df.loc[:, 'weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df.loc[:, 'weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(HTML(interactions_df.head(15).to_html()))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[0-160],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[160-260],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[160-260],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several version of the prepare_users_df method and test which works best in your recommender.

In [96]:
#Not used for now
# def n_people_bucket_label (row):
#     if row['n_people_bucket'] =='[1-1]':
#         return 1
#     elif row['n_people_bucket'] =='[2-2]':
#         return 2
#     elif row['n_people_bucket'] =='[3-4]':
#         return 3.5
#     return 5

# def  room_segment_label(row):
#     if row['room_segment'] =='[0-160]':
#         return 1
#     elif row['room_segment'] =='[160-260]':
#         return 2
#     elif row['room_segment'] =='[260-360]':
#         return 3
#     elif row['room_segment'] =='[360-500]':
#         return 4
#     elif row['room_segment'] =='[500-900]':
#         return 5
#     return 6

# def length_of_stay_bucket_label(row):
#     if row['length_of_stay_bucket'] =='[0-1]':
#         return 1 
#     elif row['length_of_stay_bucket'] =='[2-3]':
#         return 2
#     elif row['length_of_stay_bucket'] =='[4-7]':
#         return 3
#     return 4

In [91]:
def prepare_users_df(interactions_df):
    
    users_df = interactions_df.drop(['item_id'], axis=1)
    users_df = pd.get_dummies(users_df, columns=['term','length_of_stay_bucket',"rate_plan","weekend_stay","room_segment","n_people_bucket"])
    
    #users_df['room_segment_number'] = users_df.apply(lambda row: label_race(row), axis=1)
    #users_df['n_people_bucket_number'] = users_df.apply(lambda row: n_people_bucket_label(row), axis=1)
    users_df = users_df.groupby('user_id').mean().reset_index()
    
    #well, those ones don't work anyway in improving the score
    #users_df['max_room_segment_number'] = dummed_df.groupby('user_id').max().reset_index()['room_segment_number']
    #users_df['min_room_segment_number'] = dummed_df.groupby('user_id').min().reset_index()['room_segment_number']
    
    users_df.columns = [str(col) + '_users' for col in users_df.columns]
    user_features = list(users_df.columns)
    user_features.remove('user_id_users')
    users_df.rename(columns={'user_id_users':'user_id'}, inplace=True)
    
    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(HTML(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1,2, 50, 115])].head(15).to_html()))

['term_WinterVacation_users', 'term_Easter_users', 'term_OffSeason_users', 'term_HighSeason_users', 'term_LowSeason_users', 'term_MayLongWeekend_users', 'term_NewYear_users', 'term_Christmas_users', 'length_of_stay_bucket_[0-1]_users', 'length_of_stay_bucket_[2-3]_users', 'length_of_stay_bucket_[4-7]_users', 'length_of_stay_bucket_[8-inf]_users', 'rate_plan_Standard_users', 'rate_plan_Nonref_users', 'weekend_stay_True_users', 'weekend_stay_False_users', 'room_segment_[0-160]_users', 'room_segment_[160-260]_users', 'room_segment_[260-360]_users', 'room_segment_[360-500]_users', 'room_segment_[500-900]_users', 'n_people_bucket_[1-1]_users', 'n_people_bucket_[2-2]_users', 'n_people_bucket_[3-4]_users', 'n_people_bucket_[5-inf]_users']


,user_id,term_WinterVacation_users,term_Easter_users,term_OffSeason_users,term_HighSeason_users,term_LowSeason_users,term_MayLongWeekend_users,term_NewYear_users,term_Christmas_users,length_of_stay_bucket_[0-1]_users,length_of_stay_bucket_[2-3]_users,length_of_stay_bucket_[4-7]_users,length_of_stay_bucket_[8-inf]_users,rate_plan_Standard_users,rate_plan_Nonref_users,weekend_stay_True_users,weekend_stay_False_users,room_segment_[0-160]_users,room_segment_[160-260]_users,room_segment_[260-360]_users,room_segment_[360-500]_users,room_segment_[500-900]_users,n_people_bucket_[1-1]_users,n_people_bucket_[2-2]_users,n_people_bucket_[3-4]_users,n_people_bucket_[5-inf]_users
0,1,0.130435,0.0,0.652174,0.086957,0.130435,0.000000,0.000000,0.000000,0.000000,0.608696,0.391304,0.000000,0.521739,0.478261,0.782609,0.217391,0.739130,0.130435,0.043478,0.043478,0.043478,0.000000,0.739130,0.173913,0.086957
1,2,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
47,50,0.043478,0.0,0.434783,0.304348,0.217391,0.000000,0.000000,0.000000,0.000000,0.913043,0.086957,0.000000,0.260870,0.739130,0.782609,0.217391,0.217391,0.260870,0.304348,0.217391,0.000000,0.000000,0.173913,0.521739,0.304348
92,96,0.083333,0.0,0.708333,0.125000,0.041667,0.041667,0.000000,0.000000,0.250000,0.666667,0.041667,0.041667,0.291667,0.708333,0.750000,0.250000,0.666667,0.208333,0.083333,0.041667,0.000000,0.041667,0.333333,0.541667,0.083333
111,115,0.727273,0.0,0.272727,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.363636,0.136364,0.000000,1.000000,0.000000,0.363636,0.636364,0.954545,0.045455,0.000000,0.000000,0.000000,0.818182,0.090909,0.045455,0.045455
675,706,0.091988,0.0,0.451039,0.189911,0.207715,0.038576,0.011869,0.008902,0.169139,0.459941,0.272997,0.097923,0.994065,0.005935,0.676558,0.323442,0.893175,0.077151,0.011869,0.011869,0.000000,0.041543,0.094955,0.738872,0.124629
1699,1736,0.034483,0.0,0.482759,0.206897,0.275862,0.000000,0.000000,0.000000,0.241379,0.551724,0.206897,0.000000,0.172414,0.827586,0.448276,0.551724,0.310345,0.517241,0.137931,0.034483,0.000000,0.379310,0.413793,0.206897,0.000000
7639,7779,0.037037,0.0,0.296296,0.259259,0.370370,0.000000,0.000000,0.037037,0.111111,0.296296,0.481481,0.111111,1.000000,0.000000,0.814815,0.185185,0.888889,0.074074,0.037037,0.000000,0.000000,0.000000,0.037037,0.740741,0.222222


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into on-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because of the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features.

In [7]:
def prepare_items_df(interactions_df):
    
    #interactions_df = interactions_df.groupby(["item_id"]).mean().reset_index()#.agg(lambda x:x.value_counts(["term", "room_segment","rate_plan","weekend_stay","n_people_bucket"]).index[0])
    interactions_df = interactions_df.drop(['user_id'], axis=1, errors='ignore')
    items_df = pd.get_dummies(interactions_df)#, columns=['term','length_of_stay_bucket', 'room_segment',"rate_plan","weekend_stay","n_people_bucket"])
    items_df = items_df.groupby('item_id').mean().reset_index()
    items_df.columns = ['items_' + str(col) for col in items_df.columns]
    item_features = list(items_df.columns)
    item_features.remove('items_item_id')
    items_df.rename(columns={'items_item_id':'item_id'}, inplace=True)
    return items_df, item_features



items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(HTML(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15).to_html()))

['items_term_WinterVacation', 'items_term_Easter', 'items_term_OffSeason', 'items_term_HighSeason', 'items_term_LowSeason', 'items_term_MayLongWeekend', 'items_term_NewYear', 'items_term_Christmas', 'items_length_of_stay_bucket_[0-1]', 'items_length_of_stay_bucket_[2-3]', 'items_length_of_stay_bucket_[4-7]', 'items_length_of_stay_bucket_[8-inf]', 'items_rate_plan_Standard', 'items_rate_plan_Nonref', 'items_room_segment_[0-160]', 'items_room_segment_[160-260]', 'items_room_segment_[260-360]', 'items_room_segment_[360-500]', 'items_room_segment_[500-900]', 'items_n_people_bucket_[1-1]', 'items_n_people_bucket_[2-2]', 'items_n_people_bucket_[3-4]', 'items_n_people_bucket_[5-inf]', 'items_weekend_stay_True', 'items_weekend_stay_False']


,item_id,items_term_WinterVacation,items_term_Easter,items_term_OffSeason,items_term_HighSeason,items_term_LowSeason,items_term_MayLongWeekend,items_term_NewYear,items_term_Christmas,items_length_of_stay_bucket_[0-1],items_length_of_stay_bucket_[2-3],items_length_of_stay_bucket_[4-7],items_length_of_stay_bucket_[8-inf],items_rate_plan_Standard,items_rate_plan_Nonref,items_room_segment_[0-160],items_room_segment_[160-260],items_room_segment_[260-360],items_room_segment_[360-500],items_room_segment_[500-900],items_n_people_bucket_[1-1],items_n_people_bucket_[2-2],items_n_people_bucket_[3-4],items_n_people_bucket_[5-inf],items_weekend_stay_True,items_weekend_stay_False
0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0
1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0
2,2,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
3,3,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0
4,4,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0
5,5,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0
6,6,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object for later use in the recommend method. Overwrite the users_df variable. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real interactions and equal to 0 for those newly added interactions. Generate several negative interactions per every positive interactions (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and use initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, Neural Networks or any model of your choice.
  - A different approach where you treat each item as a class, you train directly on categorical features of items and users (you would have to design appropriate categorical features for users) and you fit classifiers (e.g. Decision Tree classifier, Naive Bayes classifier etc.) instead of regressors.

In [94]:
import random
import itertools
def fill_negatives(interactions_df, negative_interactions, n_neg_per_pos):  
    arr = interactions_df[['user_id', 'item_id']].to_numpy()
    how_long = n_neg_per_pos * len(interactions_df)
    
    while len(negative_interactions) < how_long:
        a = random.randint(0,interactions_df.item_id.max())
        b = random.randint(0,interactions_df.user_id.max())
        if not([a,b] == arr).all(axis = 1).any():
            negative_interactions.append((a,b,0))
    return negative_interactions

In [95]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
       
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []
        

        negative_interactions = fill_negatives(interactions_df, negative_interactions, self.n_neg_per_pos)

        
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values
    
        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        self.recommender_df = self.recommender_df[:0]
        
        
        users_df = pd.merge(self.users_df, users_df)
        items_df, features = prepare_items_df(items_df)
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            records = []
            user_one = user.to_frame().T
            cart_list = pd.merge(user_one,items_df, how='cross')
            cart_list_predict = cart_list.drop(['item_id', 'user_id', 'items_user_id'], axis=1, errors='ignore')
            scores = self.model.predict(cart_list_predict)
            
            chosen_ids = None
            ids = (-scores).argsort()[:n_recommendations]
            chosen_ids = cart_list.iloc[ids]['item_id']

            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [11]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [12]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [13]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(HTML(recommendations.to_html()))

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,92,1.000000,OffSeason,[4-7],Nonref,[0-160],[2-2],True
1,1.0,30,0.999655,OffSeason,[4-7],Standard,[0-160],[2-2],True
2,1.0,100,0.998437,OffSeason,[4-7],Nonref,[160-260],[2-2],True
3,1.0,86,0.998333,OffSeason,[4-7],Nonref,[0-160],[2-2],False
4,1.0,37,0.998333,OffSeason,[4-7],Standard,[0-160],[2-2],False
5,1.0,36,0.998333,OffSeason,[4-7],Standard,[0-160],[3-4],False
6,1.0,146,0.998333,OffSeason,[4-7],Nonref,[0-160],[3-4],False
7,1.0,87,0.998093,OffSeason,[4-7],Standard,[160-260],[2-2],True
8,1.0,106,0.997449,OffSeason,[4-7],Nonref,[0-160],[3-4],True
9,1.0,85,0.997332,OffSeason,[2-3],Nonref,[0-160],[2-2],True


# Tuning method

In [14]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [15]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(HTML(results.to_html()))
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [100]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [101]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [102]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [103]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [91]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 2),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 10),
    'max_depth': hp.quniform('max_depth', 2, 10, 2),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 3),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=50, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 50/50 [58:07<00:00, 69.75s/trial, best loss: -0.03277345350266284] 


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.012833,0.02435,0.028628,0.042448,0.012833,0.019626,0.021381,0.025779


Best parameters:
{'learning_rate': 0.09441000320861691, 'max_depth': 4.0, 'min_samples_split': 21.0, 'n_estimators': 280.0, 'n_neg_per_pos': 10.0}


# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [92]:
cb_user_item_recommender = SVRCBUIRecommender(
    **{'C': 0.3576418626772324, 'n_neg_per_pos': 10.0})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
SVRCBUIRecommender_tts_results = [['SVRCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

SVRCBUIRecommender_tts_results = pd.DataFrame(
    SVRCBUIRecommender_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(SVRCBUIRecommender_tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRCBUIRecommender,0.014808,0.025666,0.033235,0.049029,0.014808,0.020797,0.023898,0.028939


In [93]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(amazon_tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.029286,0.077328,0.11155,0.164528,0.029286,0.056969,0.070972,0.087829


In [99]:
tts_results = pd.concat([SVRCBUIRecommender_tts_results, amazon_tts_results]).reset_index(drop=True)
display(HTML(tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRCBUIRecommender,0.014808,0.025666,0.033235,0.049029,0.014808,0.020797,0.023898,0.028939
1,AmazonRecommender,0.029286,0.077328,0.111550,0.164528,0.029286,0.056969,0.070972,0.087829
